In [4]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-ada-002"
)

print(response.data[0].embedding)

[-0.0069986856542527676, -0.005245656240731478, 0.011908512562513351, -0.024931978434324265, -0.02470361441373825, 0.03995027020573616, -0.010148766450583935, -0.009356209076941013, -0.013218247331678867, -0.009759203530848026, -0.011834629811346531, 0.007871842011809349, -0.01421230286359787, 0.007750943768769503, 0.010041301138699055, -0.005134832579642534, 0.022822970524430275, -0.0015758797526359558, 0.015031726099550724, -0.010121899656951427, 0.004785569850355387, 0.012479422613978386, 0.00480571947991848, 0.010672660544514656, -0.00644456734880805, -0.0004743591125588864, 0.005588202271610498, -0.012553305365145206, 0.016281012445688248, 0.004456456750631332, 0.00668636430054903, -0.0069449529983103275, -0.015139191411435604, -0.006689722649753094, -0.01872584968805313, 0.004103836137801409, 0.0033196741715073586, -0.018981080502271652, 0.030520180240273476, -0.0075897458009421825, 0.00818080548197031, 0.009510690346360207, -0.0010922852670773864, -0.00045462913112714887, -0.008

In [27]:
import pandas as pd
import tiktoken
from openai import OpenAI
client = OpenAI()

embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

# load & inspect dataset
input_datapath = "cleandata.csv"
df = pd.read_csv(input_datapath)
df = df[["isbn13", "title", "authors", "categories", "description"]]
df = df.dropna()
df["combined"] = (
    "Authors: "+ df.authors.str.strip() + 
    "; Title: " + df.title.str.strip() + 
    "; Categories: " + df.categories.str.strip() + 
    "; Description: " + df.description.str.strip()
)
df.head(2)



,isbn13,title,authors,categories,description,combined
0,9780002005883,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,Authors: Marilynne Robinson; Title: Gilead; Ca...
1,9780002261982,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,Authors: Charles Osborne;Agatha Christie; Titl...


In [28]:
def get_embedding(text, model=embedding_model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("output/book_description_with_embeddings.csv")

In [10]:
print(type(df['embedding'][0]))

<class 'numpy.ndarray'>


In [29]:
# read data from csv
import pandas as pd
import numpy as np

df = pd.read_csv('output/book_description_with_embeddings.csv')
df['embedding'] = df.embedding.apply(eval).apply(np.array)

In [31]:
print(type(df['embedding'][0]))

<class 'numpy.ndarray'>


In [32]:
# Define the function to get embeddings from OpenAI
def get_embedding(text, model=embedding_model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

# Define the function to compute cosine similarity
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Define the search_reviews function
def search_reviews(df, book_description, embedding_model=embedding_model, n=3, pprint=True):
    embedding = get_embedding(book_description, model=embedding_model)
    df['similarities'] = df['embedding'].apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(n)
    return res
    
res = search_reviews(df, 'Shakespeare', n=5)
print(res)

      Unnamed: 0         isbn13                                    title  \
4254        4366  9780743484961                            Twelfth Night   
4251        4363  9780743482776                            Twelfth Night   
4327        4440  9780748702558                         Romeo and Juliet   
6344        6533  9781903436578                    Shakespeare's Sonnets   
1237        1251  9780199267170  William Shakespeare: The Complete Works   

                  authors categories  \
4254  William Shakespeare      Drama   
4251  William Shakespeare      Drama   
4327  William Shakespeare      Drama   
6344  William Shakespeare      Drama   
1237  William Shakespeare      Drama   

                                            description  \
4254  Background information about Shakespeare, Eliz...   
4251  Background information about Shakespeare, Eliz...   
4327  Presents the original text of Shakespeare's pl...   
6344                                 No Marketing Blurb   
1237  P